# BCI Testbed for FATES

The purpose of this testbed code is to compare FATES output to observations at BCI, and also to understand some of the internal paramteric control over FATES predictions.  The overall order of things is to first load an ensemble of FATES model inputs, then look at several metrics of model predictions as projected onto the trait covariance matrices that were used to generate the model ensemble, and well as looking at several model predictions as compared to observed ecosystem states and fluxes at the BCI site.

#### First we will load all the libraries that we need

In [ ]:
import netCDF4 as nc4
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import dateutil.parser
import datetime
import map_funcs
from IPython.display import Image
from scipy import interpolate




import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

# set the plot style
plt.style.use('seaborn-ticks')
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5, "lines.markersize": 6.0, 'lines.markeredgewidth': 1.0})
#sns.set_context("poster")

print(sns.__version__)

# First steps: load the data and plot some basic properties


Load all the FATES history output that was generated by the model ensemble, as well as the traits matrix input to the model.


In [ ]:
### uncomment this for bare-ground runs
filename_history = '/Users/cdkoven/datasets/fates_output/bci_testbed/fates_clm5_fullmodel_bci_parameter_ensemble_1pft_190329_multiinst_576inst_b9c92b7_106ac7a.h0.ensemble.sofar.nc'

## now load the size-structured data.  it is on a second history tape with an annual write frequency
filename_history2 = '/Users/cdkoven/datasets/fates_output/bci_testbed/fates_clm5_fullmodel_bci_parameter_ensemble_1pft_190329_multiinst_576inst_b9c92b7_106ac7a.h1.ensemble.sofar.nc'

# ### uncomment this for inventory runs
# filename_history = '/Users/cdkoven/datasets/fates_output/bci_testbed/fates_clm5_fullmodel_bci_parameter_ensemble_1pft_v004inv_b1bf522_819609f.h0.ensemble.sofar.nc'

filename_params = '/Users/cdkoven/datasets/fates_output/bci_testbed/fates_params_default_106ac7a_mod1PFT_exp1.ensemble.c190329.nc'

histfile = nc4.Dataset(filename_history)
paramfile = nc4.Dataset(filename_params)
histfile2 = nc4.Dataset(filename_history2)

traits_matrix = np.loadtxt('traits_matrix_used_in_param_files_190329.txt')


#### Next lets look at which ensemble members predict a surviving forest.  Calculate an index of survival and project that back on to the trait covariance matrix.

In [ ]:
biomass_var = histfile.variables['AGB']

nens = biomass_var.shape[0]
nts = biomass_var.shape[1]
nyears = nts/12
print(nens)

startyear = nyears-50
endyear = nyears-1


survival_threshold = 100.
survived = biomass_var[:,nts-1,0] > survival_threshold
print(' fraction of ensemble members with surviving forests = '+str(survived.sum()/float(nens)))


#column_names = ['Vcmax25', 'wood_dens', 'LMA', 'N/area','leaf_life','b_mort','crown_dbh_coef','crown_dbh_exp','bleaf_dbh','fineroot_leaf','npp_repro','cstarv_mort']
column_names = ['Vcmax25top', 'Wood Dens.', 'LMA', 'Leaf N/area','Leaf Lifespan','Back. Mort.','Cr. Area Coef','Cr. Area Exp','Bleaf Allom.','Fine Root:Leaf','NPP Repr.','C Starv Mort.']

ncols = len(column_names)
print(ncols)
print(traits_matrix.shape)
print(len(survived))

df_traits = pd.DataFrame(traits_matrix, columns=column_names)

columns_log = [0,2,3,4,5,11]
column_names_log = ['log Vcmax25', 'wood_dens', 'log LMA', 'log N/area','log leaf_life','log b_mort','crown_dbh_coef','crown_dbh_exp','bleaf_dbh','fineroot_leaf','npp_repro','log cstarv_mort']
traits_matrix_partiallylog = traits_matrix.copy()
for i in range(len(columns_log)):
    traits_matrix_partiallylog[:,columns_log[i]] = np.ma.log10(traits_matrix_partiallylog[:,columns_log[i]])
df_traits_log = pd.DataFrame(traits_matrix_partiallylog, columns=column_names_log)

#colors = 1. + 10. * survived
#pd.plotting.scatter_matrix(df_traits, alpha=0.7, figsize=(30,30), diagonal='hist', c=colors, s=80)

df_traits_log["survival"] = pd.cut(survived, bins=[-1,0.5,2.], labels=['Dead','Alive'])

survivalplot = sns.pairplot(df_traits_log, hue="survival",
                 hue_order=df_traits_log.survival.cat.categories,
                 palette="YlGnBu", diag_kind="hist")

#### Next lets plot a histogram of mean LAI values

In [ ]:
mean_lai = histfile.variables['ELAI'][:,startyear*12:endyear*12,0].mean(axis=1)
lai_series = pd.Series(mean_lai, name='LAI')
sns.distplot(lai_series, kde=False, rug=False, color="k")

#### Plot a joint ditribution of LAI and GPP values.

In [ ]:
mean_gpp = np.ma.masked_array(histfile.variables['GPP'][:,startyear*12:endyear*12,0].mean(axis=1) * 86400 *365, mask=np.logical_not(survived))
gpp_series = pd.Series(mean_gpp, name='GPP')
sns.jointplot(x=lai_series, y=gpp_series, color="k");


#### Let's now make a plot of biomass trajectories, to see where we are at in terms of steady-state and the overall progression of biomass accumulation across the ensemble members.

In [ ]:
sns.set_context("poster")
biomass_trajectory_annual = biomass_var[:,::12,0]
biomass_trajectory_annual.shape
time = np.arange(biomass_trajectory_annual.shape[1])
biomass_df = pd.DataFrame(biomass_trajectory_annual)
 
fig=plt.figure(figsize=(15, 7.5))

# multiple line plot
for column in range(nens):
    plt.plot(time, biomass_trajectory_annual[column,:]*1e-3, marker='', linewidth=0.25, alpha=0.9, color='green')
 
# Add titles
#plt.title("Timeseries of biomass", loc='left')
plt.xlabel("Time (years since start of run)")
plt.ylabel("Aboveground Biomass (kg C / m$^2$)")

biomass_observed = 13.6  ## table S2 of Meakem et al., 2018.
plt.plot(np.array([0.,200]), np.array([biomass_observed,biomass_observed]), color='black')

plt.savefig('biomass_timeseries.pdf')



# Understanding some parametric control

Now we will plot several instances of the trait covariance matrix as colored by various model predictions, in order to understand how the paramters govern various model predictions.

#### Plot the trait covariance matrix as colored by the final LAI

In [ ]:
df_traits_log["mean_LAI"] = pd.cut(mean_lai, bins=np.arange(10))

laiplot = sns.pairplot(df_traits_log, hue="mean_LAI",
                 hue_order=df_traits_log.mean_LAI.cat.categories,
                 palette="YlGnBu")

laiplot.savefig('lai_pairplot.pdf', format='pdf')

#### Next, plot the trait covariance matrix as colored by final mean GPP

In [ ]:
df_traits_log["mean_gpp"] = pd.cut(mean_gpp, bins=np.arange(10)*400.)


gppscatterplot = sns.pairplot(df_traits_log, hue="mean_gpp",
                 hue_order=df_traits_log.mean_gpp.cat.categories,
                 palette="YlGnBu")

gppscatterplot.savefig('gpp_pairplot.pdf', format='pdf')

#### Next, let's  plot the carbon use efficieny of the runs, as projected onto the trait covariance matrix.  Here I am defining CUE as the ratio of NPP/GPP.

In [ ]:
mean_npp = histfile.variables['NPP'][:,startyear*12:endyear*12,0].mean(axis=1) * 86400 *365

mean_cue = np.ma.masked_array(mean_npp[:] / mean_gpp[:], mask=np.logical_not(survived))

df_traits_log["mean_cue"] = pd.cut(mean_cue, bins=np.arange(11)*.1)


cuescatterplot = sns.pairplot(df_traits_log, hue="mean_cue",
                 hue_order=df_traits_log.mean_cue.cat.categories,
                 palette="YlGnBu", dropna=True)

#### Plot autotrophic respiration itself, projected onto TCM

In [ ]:
mean_ar = histfile.variables['AR'][:,startyear*12:endyear*12,0].mean(axis=1) * 86400 *365

df_traits_log["mean_ar"] = pd.cut(mean_ar, bins=np.arange(10)*200.)


cuescatterplot = sns.pairplot(df_traits_log, hue="mean_ar",
                 hue_order=df_traits_log.mean_ar.cat.categories,
                 palette="YlGnBu")

#### The same thing , now for latent heat fluxes

In [ ]:
mean_transp = np.ma.masked_array(histfile.variables['FCTR'][:,startyear*12:endyear*12,0].mean(axis=1))
mean_grndevap = np.ma.masked_array(histfile.variables['FGEV'][:,startyear*12:endyear*12,0].mean(axis=1))
mean_canevap = np.ma.masked_array(histfile.variables['FCEV'][:,startyear*12:endyear*12,0].mean(axis=1))

mean_lh = mean_transp + mean_grndevap + mean_canevap
print(mean_lh.max())
df_traits_log["mean_lh"] = pd.cut(mean_lh, bins=np.arange(10)*10.)
transpscatterplot = sns.pairplot(df_traits_log, hue="mean_lh",
                 hue_order=df_traits_log.mean_lh.cat.categories,
                 palette="YlGnBu")

#### Now we will look at the final biomass as projected onto the TCM

In [ ]:
final_biomass = biomass_trajectory_annual[:,startyear:endyear].mean(axis=1) * 1e-3
print(final_biomass.max())
df_traits_log["final_biomass"] = pd.cut(final_biomass, bins=np.arange(10)*5.)
biomassscatterplot = sns.pairplot(df_traits_log, hue="final_biomass",
                 hue_order=df_traits_log.final_biomass.cat.categories,
                 palette="YlGnBu")

#### Crown area index, which is the total area of plant canopies per area ground

In [ ]:
crown_area_index = histfile2.variables['CANOPY_AREA_BY_AGE'][:,startyear:endyear,:,0].mean(axis=1).sum(axis=1)

df_traits_log["final_crown_area_index"] = pd.cut(crown_area_index, bins=np.arange(11)*.3)
final_crown_area_index_scatterplot = sns.pairplot(df_traits_log, hue="final_crown_area_index",
                 hue_order=df_traits_log.final_crown_area_index.cat.categories,
                 palette="YlGnBu")

# Comparison of FATES against observational datasets
## Eddy-Covariance Data

#### Now let's compare the seasonal cycles of GPP, LH, and SH against flux tower observations. First load and process the eddy covariance data.

In [ ]:
bci_fluxtower_datafilename = 'benchmark_datasets/BCI_v3.1.csv'

bci_fluxtower_data = np.genfromtxt(bci_fluxtower_datafilename, dtype={'names': ('date','tair','RH','vpd','p_kpa','PPT','Rs','Rs_dn','Rl_dn','Rl_up','Rnet','LE','H','Par_tot','Par_diff','SWC','ubar','ustar','WD','gpp','FLAG'),'formats': ('S16','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4')}, delimiter=',', skip_header=2)

ntim = len(bci_fluxtower_data)

time_start = datetime.datetime(2010, 1, 1, 0, 00)

tdelt = np.ma.masked_all(ntim)
month = np.ma.masked_all(ntim, dtype=np.int)
year = np.ma.masked_all(ntim, dtype=np.int)
gpp = np.ma.masked_all(ntim)
LE = np.ma.masked_all(ntim)
H = np.ma.masked_all(ntim)

for i in range(ntim):
    ts = dateutil.parser.parse(bci_fluxtower_data[i][0])
    tdelt[i] = (ts - time_start).days + (ts - time_start).seconds / 86400.
    month[i] = ts.month
    year[i] = ts.year
    gpp[i] = bci_fluxtower_data[i][19]
    LE[i] = bci_fluxtower_data[i][11]
    H[i] = bci_fluxtower_data[i][12]

H_masked = np.ma.masked_invalid(H)

nyears = (year.max() - year.min()) + 1
nmonths = nyears * 12

gpp_monthly = np.ma.masked_all(nmonths)
gpp_monthyear = np.ma.masked_all([nyears,12])
LE_monthyear = np.ma.masked_all([nyears,12])
H_monthyear = np.ma.masked_all([nyears,12])

for i in range(nyears):
    for j in range(12):
        mask = (year[:] == (year.min() + i)) * (month[:] == (j + 1))
        index = i*12 + j
        # gpp_monthly2[index] = gpp[mask].mean()                                                                                                                                                                                                                              
        if mask.sum() > 0:
            gpp_monthly[index] = (gpp * mask).sum() / mask.sum()
            gpp_monthyear[i,j] = gpp_monthly[index]
            LE_monthyear[i,j] = (LE * mask).sum() / mask.sum()
        H_monthyear[i,j] = H_masked[mask].mean()

months = np.arange(12)

### change gpp units to grams or carbon per meter squared per year
gpp_gcm2y = gpp_monthyear * 1e-6 * 12.0107 * (86400 * 365.25)

## write monthly gpp as a text file
np.savetxt('benchmark_datasets/gpp_gcm2y',gpp_gcm2y)

#### Now lets plot seasonal cycles of GPP from FATES and the tower data together.

In [ ]:
# calculate sesonal cycle of GPP from FATES ensembles
gpp_seascycle = np.ma.masked_all([nens, 12])
offset = 7  # first month is actually June, so I need to add this to get to January

for i in range(12):
    gpp_seascycle[:,i] = histfile.variables['GPP'][:,offset+i+(startyear)*12:offset+i+endyear*12:12,0].mean(axis=1) * 86400 *365

fig=plt.figure(figsize=(10, 5))

plt.plot(months, gpp_seascycle.transpose(), linewidth=0.15, alpha=0.9, color='green')

plt.plot(months, gpp_gcm2y.transpose(), marker='', linewidth=1.0, alpha=0.9, color='blue')


plt.title("Seasonal cycle of GPP", loc='left')
plt.xlabel("Month")
plt.ylabel("GPP (g C / m$^2$ / y)")

plt.text(7, 5200, 'Observations', color ='blue', fontsize=15)
plt.text(7, 4900, 'FATES ensemble members', color ='green', fontsize=15)

plt.savefig('gpp_seascycle.pdf')

#### Now lets plot sesaonal cycles of LH from FATES and the tower data together

In [ ]:
# calculate sesonal cycle of LH from FATES ensembles
LH_seascycle = np.ma.masked_all([nens, 12])

for i in range(12):
    LH_seascycle[:,i] = histfile.variables['FCTR'][:,offset+i+(startyear)*12:offset+i+endyear*12:12,0].mean(axis=1)
    +histfile.variables['FGEV'][:,offset+i+(startyear)*12:offset+i+endyear*12:12,0].mean(axis=1)
    +histfile.variables['FCEV'][:,offset+i+(startyear)*12:offset+i+endyear*12:12,0].mean(axis=1)
    
fig=plt.figure(figsize=(10, 5))

plt.plot(months, LH_seascycle.transpose(), linewidth=0.15, alpha=0.9, color='green')

plt.plot(months, LE_monthyear.transpose(), marker='', linewidth=1.0, alpha=0.9, color='blue')


plt.title("Seasonal cycle of LH", loc='left')
plt.xlabel("Month")
plt.ylabel("LH (W/m$^2$)")


plt.text(7, 90, 'Observations', color ='blue', fontsize=15)
plt.text(7, 83, 'FATES ensemble members', color ='green', fontsize=15)
plt.savefig('lh_seascycle.pdf')

#### Now lets plot seasonal cycles of SH from FATES and the tower data together

In [ ]:
# calculate sesonal cycle of SH from FATES ensembles
SH_seascycle = np.ma.masked_all([nens, 12])

for i in range(12):
    SH_seascycle[:,i] = histfile.variables['FSH'][:,offset+i+(startyear)*12:offset+i+endyear*12:12,0].mean(axis=1)

fig=plt.figure(figsize=(10, 5))

plt.plot(months, SH_seascycle.transpose(), linewidth=0.15, alpha=0.9, color='green')

plt.plot(months, H_monthyear.transpose(), marker='', linewidth=1.0, alpha=0.9, color='blue')


plt.title("Seasonal cycle of SH", loc='left')
plt.xlabel("Month")
plt.ylabel("SH (W/m$^2$)")


plt.text(7, 97, 'Observations', color ='blue', fontsize=15)
plt.text(7, 90, 'FATES ensemble members', color ='green', fontsize=15)
plt.savefig('sh_seascycle.pdf')

In [ ]:
### load the LAI data and plot the data on the same joint distribution plot as the model ensemble
lai_datafilename = 'benchmark_datasets/LAI.csv'
lai_data = pd.read_csv(lai_datafilename)

laimean = np.array([lai_data['LAI '][:].mean()])
laistd = np.array([lai_data['LAI '][:].std()])
gppmean = np.array([gpp_gcm2y.mean()])
gppstd = np.array([gpp_gcm2y.std()])
angle = np.array([0.])

lai_ens = lai_series.values
gpp_ens = gpp_series.values

map_funcs.xyplot(lai_ens, gpp_ens, dots=True, xtitle='LAI', ytitle='GPP(gC m~S~-2~N~ y~S~-1~N~)', showjupyter=True, file='lai_v_gpp', overlay_ellipses_x=laimean, overlay_ellipses_y=gppmean, overlay_ellipses_xaxis=laistd, overlay_ellipses_yaxis=gppstd, overlay_ellipses_angle=angle, overlay_ellipses_color='black', overlay_ellipses_filled=True, overlay_ellipses_opacity=0.3)
laistd

## Census Data

We want to load the size-structured FATES output data and compare this to BCI census data.

#### First load the raw size distribution data.  Plot as number density in log/log space

In [ ]:
nplant_scls = histfile2.variables['NPLANT_SCLS']
fates_levscls = histfile2.variables['fates_levscls'][:]

nlevscls= len(fates_levscls)

levscls_ext = np.append(fates_levscls,200.)
dlevscls = levscls_ext[1:] - levscls_ext[0:len(levscls_ext)-1]
levscls_mid = fates_levscls + dlevscls/2.
levscls_ext_logscale = levscls_ext.copy()
levscls_ext_logscale[0] = .5

nplant_scls_timeave = nplant_scls[:,startyear:endyear,:,0].mean(axis=1)

nplant_scls_timeave_n_percm =  nplant_scls_timeave/dlevscls

## open BCI inventory data plot                                                                                                                                 
filename_bci_inventory = 'benchmark_datasets/census_bmks_lscaled_allyears_1pft_v4_25scbci_181101.nc'
fin_bci_inv = nc4.Dataset(filename_bci_inventory)

## load size distributions from census data
sizedist_inv = fin_bci_inv.variables['abund_by_size_census'][:,:,1]
sizedist_inv_timeave = sizedist_inv[:,:].mean(axis=0)

# get error on obs and ready that too                                                                                                                           
sizedist_inv_error_ll = np.ma.masked_all([2,len(levscls_mid)-1])
sizedist_inv_error_ul = np.ma.masked_all([2,len(levscls_mid)-1])
sizedist_inv_error_ll[1,:] = fin_bci_inv.variables['abund_by_size_census'][:,:,0].min(axis=0)/dlevscls[1:]
sizedist_inv_error_ul[1,:] = fin_bci_inv.variables['abund_by_size_census'][:,:,2].max(axis=0)/dlevscls[1:]

fig=plt.figure(figsize=(10, 10))

for column in range(nens):
    plt.loglog(levscls_mid[1:], nplant_scls_timeave_n_percm[column,1:], marker='', linewidth=0.25, alpha=0.9, color='green')

plt.loglog(levscls_mid[1:], sizedist_inv[:].mean(axis=0)/dlevscls[1:], marker='', linewidth=1.5, alpha=0.9, color='blue')

# Add titles
plt.title("Tree Size Distributions", loc='left')
plt.xlabel("Tree Diameter (cm)")
plt.ylabel("Tree number density (n/ha/cm)")
plt.ylim(ymin=1e-2)


plt.text(30, 1e4, 'Observations', color ='blue', fontsize=15)
plt.text(30, 0.6e4, 'FATES ensemble members', color ='green', fontsize=15)
plt.savefig('sizedist.pdf')

#### Plot growth rates, conditional on size

In [ ]:
## read census data
growth_rate_inv = fin_bci_inv.variables['growth_increment_by_size_census'][:,:,1]
dclass_inv = fin_bci_inv.variables['dclass'][:]
ddbh_inv_error_ll = np.ma.masked_all([2,len(levscls_mid)])
ddbh_inv_error_ul = np.ma.masked_all([2,len(levscls_mid)])
ddbh_inv_error_ll[1,1:] = fin_bci_inv.variables['growth_increment_by_size_census'][1:,:,0].min(axis=0)
ddbh_inv_error_ul[1,1:] = fin_bci_inv.variables['growth_increment_by_size_census'][1:,:,2].max(axis=0)

## FATES
ddbh_scls = (histfile2.variables['DDBH_CANOPY_SCLS'][:,startyear:endyear,:,0].mean(axis=1) 
             + histfile2.variables['DDBH_UNDERSTORY_SCLS'][:,startyear:endyear,:,0].mean(axis=1)
            )/ nplant_scls_timeave

fig=plt.figure(figsize=(24, 24))

for column in range(nens):
    plt.plot(levscls_mid[1:], ddbh_scls[column,1:], marker='', linewidth=0.25, alpha=0.9, color='green')

plt.plot(levscls_mid[1:], growth_rate_inv[:].mean(axis=0), marker='', linewidth=1.5, alpha=0.9, color='blue')


# Add titles
plt.title("Tree Growth Rates", loc='left')
plt.xlabel("Tree Diameter (cm)")
plt.ylabel("Diameter Increment (cm/yr)")
plt.ylim(ymax=10)
plt.ylim(ymin=0)

plt.text(100, 3.6, 'Observations', color ='blue', fontsize=15)
plt.text(100, 3.4, 'FATES ensemble members', color ='green', fontsize=15)

In [ ]:
## FATES DDBH by canopy strata


nplant_canopy_scls = histfile2.variables['NPLANT_CANOPY_SCLS']
nplant_canopy_scls_timeave = nplant_canopy_scls[:,startyear:endyear,:,0].mean(axis=1)

ddbh_canopy_scls = (histfile2.variables['DDBH_CANOPY_SCLS'][:,startyear:endyear,:,0].mean(axis=1) 
            )/ nplant_canopy_scls_timeave

fig=plt.figure(figsize=(24, 24))

for column in range(nens):
    plt.plot(levscls_mid[1:], ddbh_canopy_scls[column,1:], marker='', linewidth=0.25, alpha=0.9, color='green')

# Add titles
plt.title("Canopy Tree Growth Rates", loc='left')
plt.xlabel("Tree Diameter (cm)")
plt.ylabel("Diameter Increment (cm/yr)")
plt.ylim(ymax=10)
plt.ylim(ymin=0)

plt.text(100, 3.6, 'Observations', color ='blue', fontsize=15)
plt.text(100, 3.4, 'FATES ensemble members', color ='green', fontsize=15)

#### Mortality Rates, as conditional on size distributions.

In [ ]:
## load census data
mort_rate_inv = fin_bci_inv.variables['mortality_rate_by_size_census'][:,:,1]
mort_rate_inv_mean = mort_rate_inv[1:,:].mean(axis=0)
    
mort_inv_error_ll = np.ma.masked_all([2,len(levscls_mid)])
mort_inv_error_ul = np.ma.masked_all([2,len(levscls_mid)])
mort_inv_error_ll[1,1:] = fin_bci_inv.variables['mortality_rate_by_size_census'][1:,:,0].min(axis=0)
mort_inv_error_ul[1,1:] = fin_bci_inv.variables['mortality_rate_by_size_census'][1:,:,2].max(axis=0)

## FATES
mortality_scls = (histfile2.variables['MORTALITY_CANOPY_SCLS'][:,startyear:endyear,:,0].mean(axis=1) + 
                 histfile2.variables['MORTALITY_UNDERSTORY_SCLS'][:,startyear:endyear,:,0].mean(axis=1)
                 ) / nplant_scls_timeave

fig=plt.figure(figsize=(24, 24))

for column in range(nens):
    plt.plot(levscls_mid[1:], mortality_scls[column,1:], marker='', linewidth=0.15, alpha=0.9, color='green')

plt.plot(levscls_mid[1:], mort_rate_inv_mean, marker='', linewidth=1.5, alpha=0.9, color='blue')


# Add titles
plt.title("Tree Mortality Rates", loc='left')
plt.xlabel("Tree Diameter (cm)")
plt.ylabel("Mortality Rate (1/yr)")
plt.ylim(ymax=0.4)


plt.text(100, 0.33, 'Observations', color ='blue', fontsize=15)
plt.text(100, 0.32, 'FATES ensemble members', color ='green', fontsize=15)

#### Now we want to plot recruitment rates into each size class for the model and for observations
 Calculate recruitment into each bin.  This is now an output variable of FATES.
 

In [ ]:
### load recruitment rates
## use new growthflux variables

use_new_counting_method = True

if use_new_counting_method:
    ### new way of doing it
    growthflux_fusion_scpf = histfile2.variables['GROWTHFLUX_FUSION_SCPF'][:,startyear:endyear,0:26,0].mean(axis=1)
    growthflux_scpf = histfile2.variables['GROWTHFLUX_SCPF'][:,startyear:endyear,0:26,0].mean(axis=1)
    recruitment_rate_into_bin_i = np.ma.masked_array(growthflux_fusion_scpf +growthflux_scpf)
else:
    ## this is the old way of calculating recruitment, based on continuity
    recruitment_rate_smallest = histfile2.variables['RECRUITMENT'][:,startyear:endyear,:,0].sum(axis=2).mean(axis=1)
    print((endyear - startyear))
    recruitment_rate_into_bin_i = np.zeros([nens,nlevscls])
    recruitment_rate_into_bin_i[:,0] = recruitment_rate_smallest[:]
    mort_rate = (histfile2.variables['MORTALITY_CANOPY_SCLS'][:,startyear:endyear,:,0].mean(axis=1) + 
                histfile2.variables['MORTALITY_UNDERSTORY_SCLS'][:,startyear:endyear,:,0].mean(axis=1)
                )
    for i in range(nlevscls-1):
        recruitment_rate_into_bin_i[:,i+1] = (nplant_scls[:,endyear,i,0] - nplant_scls[:,startyear,i,0])/(endyear - startyear) + recruitment_rate_into_bin_i[:,i] - mort_rate[:,i]

## load census data
recruit_rate_inv = fin_bci_inv.variables['new_recruits_by_size_census'][:,:,1]
recruit_rate_inv_mean = recruit_rate_inv[1:,:].mean(axis=0)


 

fig=plt.figure(figsize=(24, 24))

## the lower bound for trees is treated as zero, but really there aren't any zero-cm dbh trees.  setting this as an arbitrary number for now.
fates_levscls[0] = 0.1

for column in range(nens):
    plt.loglog(fates_levscls[:], recruitment_rate_into_bin_i[column,:], marker='', linewidth=0.15, alpha=0.9, color='green')
        
plt.loglog(fates_levscls[1:], recruit_rate_inv_mean, marker='', linewidth=1.5, alpha=0.9, color='blue')

#
## Add titles
plt.ylim(ymin=1e-2)
plt.title("Tree Recruitment Rates", loc='left')
plt.xlabel("Tree Diameter (cm)")
plt.ylabel("Recruitment Rate into size class (n/ha/yr)")

In [ ]:
####
#plt.ylim(ymin=1e-2)
#f_recruit = pd.DataFrame(np.concatenate((np.expand_dims(fates_levscls[:],1),recruitment_rate_into_bin_i.transpose()),axis=1))
#print(np.expand_dims(fates_levscls[:],1).shape)
#print(recruitment_rate_into_bin_i.transpose().shape)
#f_recruit.columns = ['Size'] + ['ens'] * 767
#g = sns.relplot(x="size", y="ens", kind="line", data=f_recruit)
#map_funcs.xyplot()
#f_recruit
#recruitment_rate_into_bin_i[recruitment_rate_into_bin_i[:] <= 0.].mask=True
recruitment_rate_into_bin_i = np.ma.masked_where(recruitment_rate_into_bin_i[:] <= 0., recruitment_rate_into_bin_i)
#print(recruitment_rate_into_bin_i.min())
#print(fates_levscls[:])
fates_levscls[0] = 0.1
xdata = np.repeat(np.expand_dims(fates_levscls[:],1), nens,axis=1).transpose()
#print(xdata.shape)
map_funcs.xyplot(xdata, recruitment_rate_into_bin_i, file='recrate',ylog=True,xlog=True,yrange=[1e-2,1e5],xrange=[fates_levscls[0],200],linethickness=0.01,shaded_line_data=crown_area_index, shaded_line_levels=np.arange(0.,2.1,0.2),colormap='rainbow',xtitle='Diameter (cm)',ytitle='Recruitment Rate (N/ha/yr)')
map_funcs.pdf_to_png('recrate')
Image("recrate.png")

show histograms of small-tree mortality as conditional on the number of canopy layers


In [ ]:
seedling_mortality_array = np.ma.masked_invalid(np.log10(recruitment_rate_into_bin_i[:,1]/recruitment_rate_into_bin_i[:,0]))
seedling_mortality = pd.DataFrame(seedling_mortality_array)
#print(seedling_mortality.shape)
seedling_mortality.columns = ["seedling_mort"]
seedling_mortality["crownarea"] = pd.cut(crown_area_index, bins=np.arange(11)*.2)
#sns.distplot(seedling_mortality["seedling_mort"])
#print(seedling_mortality)


#ax = sns.scatterplot(x="crownarea", y="seedling_mort", data=seedling_mortality)
ax = sns.scatterplot(x=crown_area_index, y=seedling_mortality_array)
plt.title("Seedling survival as function of canopy thickness", loc='left')
plt.xlabel("Crown Area Index")
plt.ylabel("log(Survival rate of <1cm trees)")

In [ ]:
#ax = sns.scatterplot(x="crownarea", y="seedling_mort", data=seedling_mortality)
ax = sns.scatterplot(x=mean_lai, y=seedling_mortality_array)
plt.title("Seedling survival as function of LAI", loc='left')
plt.xlabel("Leaf Area Index")
plt.ylabel("log(Survival rate of <1cm trees)")

In [ ]:
ay = sns.scatterplot(x=np.log10(df_traits["C Starv Mort."]), y=seedling_mortality_array)
plt.title("Seedling Survival as f(C starve mort)", loc='left')
plt.xlabel("cstarv_mort")
plt.ylabel("log(Survival rate of <1cm trees)")

In [ ]:
df_traits_log["seedling_mort"] = pd.cut(-seedling_mortality_array, bins=np.arange(10)*0.25)
biomassscatterplot = sns.pairplot(df_traits_log, hue="seedling_mort",
                 hue_order=df_traits_log.seedling_mort.cat.categories,
                 palette="YlGnBu")

In [ ]:
size_med = 70.
size_large = 120.

lev_med = np.argmin(np.abs(levscls_mid - size_med))
#print(lev_med)
lev_large = np.argmin(np.abs(levscls_mid - size_large))
#print(lev_large)

ratio_med_large = np.ma.masked_invalid(nplant_scls_timeave_n_percm[:,lev_large] / nplant_scls_timeave_n_percm[:,lev_med])
#print(ratio_med_large)

df_traits_log["ratio_med_large"] = pd.cut(ratio_med_large, bins=np.array([0.01,0.02,0.05,0.1,0.2,0.5,1.,2.,5.]))
biomassscatterplot = sns.pairplot(df_traits_log, hue="ratio_med_large",
                 hue_order=df_traits_log.ratio_med_large.cat.categories,
                 palette="YlGnBu")

In [ ]:
### explore some trait control on basic ecosystem rates
refsize = 20.
lev_refsize = np.argmin(np.abs(levscls_mid - refsize))

nplant_scls_canopy = histfile2.variables['NPLANT_CANOPY_SCLS']
nplant_scls_canopy_timeave = nplant_scls_canopy[:,startyear:endyear,:,0].mean(axis=1)

mortrate_canopy = np.ma.masked_invalid(histfile2.variables['MORTALITY_CANOPY_SCLS'][:,startyear:endyear,lev_refsize,0].mean(axis=1) / nplant_scls_canopy_timeave[:,lev_refsize])
mortrate_canopy.mean()
df_traits_log["mortrate_canopy"] = pd.cut(mortrate_canopy, bins=np.arange(10)*0.005)
mortrate_canopy_scatterplot = sns.pairplot(df_traits_log, hue="mortrate_canopy",
                 hue_order=df_traits_log.mortrate_canopy.cat.categories,
                 palette="YlGnBu")

mortrate_canopy_scatterplot.savefig('mortrate_canopy_pairplot.pdf', format='pdf')

In [ ]:
### explore some trait control on basic ecosystem rates

nplant_scls_understory = histfile2.variables['NPLANT_UNDERSTORY_SCLS']
nplant_scls_understory_timeave = nplant_scls_understory[:,startyear:endyear,:,0].mean(axis=1)

mortrate_understory = np.ma.masked_invalid(histfile2.variables['MORTALITY_UNDERSTORY_SCLS'][:,startyear:endyear,lev_refsize,0].mean(axis=1) / nplant_scls_understory_timeave[:,lev_refsize])

mortrate_understory.mean()
df_traits_log["mortrate_understory"] = pd.cut(mortrate_understory, bins=np.array([0.,0.01,0.02,0.05,0.1,0.2,0.5,1.]))
mortrate_understory_scatterplot = sns.pairplot(df_traits_log, hue="mortrate_understory",
                 hue_order=df_traits_log.mortrate_understory.cat.categories,
                 palette="YlGnBu")

mortrate_understory_scatterplot.savefig('mortrate_understory_pairplot.pdf', format='pdf')

In [ ]:
### explore some trait control on basic ecosystem rates

growthrate_canopy = np.ma.masked_invalid(histfile2.variables['DDBH_CANOPY_SCLS'][:,startyear:endyear,lev_refsize,0].mean(axis=1) / nplant_scls_canopy_timeave[:,lev_refsize])

growthrate_canopy.mean()
print(growthrate_canopy.shape)
df_traits_log["growthrate_canopy"] = pd.cut(growthrate_canopy, bins=np.arange(10)*0.5)
growthratescatterplot = sns.pairplot(df_traits_log, hue="growthrate_canopy",
                 hue_order=df_traits_log.growthrate_canopy.cat.categories,
                 palette="YlGnBu")

growthratescatterplot.savefig('growthrate_canopy_pairplot.pdf', format='pdf')

In [ ]:
### explore some trait control on basic ecosystem rates

growthrate_understory = np.ma.masked_invalid(histfile2.variables['DDBH_UNDERSTORY_SCLS'][:,startyear:endyear,lev_refsize,0].mean(axis=1) / nplant_scls_understory_timeave[:,lev_refsize])

growthrate_understory.mean()
print(growthrate_understory.shape)
df_traits_log["growthrate_understory"] = pd.cut(growthrate_understory, bins=np.array([0.,0.01,.02,.05,.1,.2,.5,1.,2.]))
growthrate_understory_scatterplot = sns.pairplot(df_traits_log, hue="growthrate_understory",
                 hue_order=df_traits_log.growthrate_understory.cat.categories,
                 palette="YlGnBu")

## Vertical profiles profiles of canopy and leaf height (to compare to LIDAR data)
#### First plot the vertical profile of leaf area, which does not take into account any information about the relative position of leaves with respect to each other, just where all the leaves are through the canopy.

In [ ]:
heightlevs = histfile2.variables['fates_levheight'][:]
dz = heightlevs[1]-heightlevs[0]
heightlevs_midpoint = dz/2. + heightlevs

### load LIDAR estimate of LAD from Detto et al 2015
filename_lidar_leafareadens = 'benchmark_datasets/LAD.csv'
lidar_leafareadens_file = pd.read_csv(filename_lidar_leafareadens)
#print(lidar_leafareadens_file.dtypes)
#print(lidar_leafareadens_file['Height (m)'].values)
#print(lidar_leafareadens_file['LAD'].values)


leafheight = histfile2.variables['LEAF_HEIGHT_DIST'][:,startyear:endyear,:,0].mean(axis=1) / dz

fig=plt.figure(figsize=(24, 24))

for column in range(nens):
    plt.plot(leafheight[column,:], heightlevs_midpoint[:], marker='', linewidth=0.15, alpha=0.9, color='green')

plt.plot(lidar_leafareadens_file['LAD'], 1.*lidar_leafareadens_file['Height (m)'], marker='', linewidth=1.5, alpha=0.9, color='blue')

plt.title("Leaf height distributions", loc='left')
plt.xlabel("LAI vertical density (m2/m3)")
plt.ylabel("height(m)")
plt.ylim(0.,50.)
plt.text(0.8, 47, 'Observations', color ='blue', fontsize=15)
plt.text(0.8, 45, 'FATES ensemble members', color ='green', fontsize=15)

#### Next plot the vertical profile of top canopy height, which is the top of each cohort that is in the canopy strata, so is the other extreme endmember of a height distributio that takes relative position completely into account.

In [ ]:
canopyheight = histfile2.variables['CANOPY_HEIGHT_DIST'][:,startyear:endyear,:,0].mean(axis=1)/ dz

fig=plt.figure(figsize=(12, 12))

for column in range(nens):
    plt.plot(canopyheight[column,:], heightlevs_midpoint[:], marker='', linewidth=0.15, alpha=0.9, color='green')


plt.title("Canopy height distributions", loc='left')
plt.xlabel("Canopy top vertical density (m2/m3)")
plt.ylabel("height(m)")
plt.ylim(0.,50.)
#plt.text(0.3, 47, 'Observations', color ='blue', fontsize=15)
plt.text(0.3, 45, 'FATES ensemble members', color ='green', fontsize=15)

In [ ]:
# first calculate the potential variance decomposition
vars_toassess = [mean_gpp,mean_lai,final_biomass,growthrate_canopy,growthrate_understory,mortrate_canopy,mortrate_understory]

nvars_to_assess = len(vars_toassess)
potential_variance_decomposition_data = np.zeros([nvars_to_assess,ncols])

for var_i in range(nvars_to_assess):
    for par_i in range(ncols):
        #
        good_data = np.ma.masked_invalid(vars_toassess[var_i])
        dataindices_touse = np.logical_not(np.ma.getmaskarray(good_data))
        y_unordered = good_data[dataindices_touse]
        x_unordered = traits_matrix[dataindices_touse,par_i]
        order = x_unordered.argsort()
        x = x_unordered[order]
        y = y_unordered[order]
        #
        thespline = interpolate.UnivariateSpline(x, y, s=1e10, k=3)
        spline_prediction = thespline(x)
        #
        map_funcs.xyplot(x,y,overlay_x=x,overlay_y=spline_prediction,showjupyter=True,dots=True, dotsize=0.01)
        #
        potential_variance_decomposition_data[var_i,par_i] = spline_prediction.var() / y.var()
        #


In [ ]:
# next calculate the minimum variance decomposition

minimum_variance_decomposition_data = np.zeros([nvars_to_assess,ncols])

for var_i in range(nvars_to_assess):
    for par_i in range(ncols):
        #
        good_data = np.ma.masked_invalid(vars_toassess[var_i])
        dataindices_touse = np.logical_not(np.ma.getmaskarray(good_data))
        y_unordered = good_data[dataindices_touse]
        y_unordered_residual = y_unordered.copy()
        #
        # to calculate the minimum variance, first subtract the fraction explained by all other variables
        print()
        for par_j in range(ncols):
            if par_i != par_j:
                #
                x_unordered_othervar = traits_matrix[dataindices_touse,par_j]
                order_othervar = x_unordered_othervar.argsort()
                order_othervar_togetback = order_othervar.argsort()
                #
                x_othervar = x_unordered_othervar[order_othervar]
                y_othervar = y_unordered_residual[order_othervar]                
                #
                thespline_othervar = interpolate.UnivariateSpline(x_othervar, y_othervar, s=1e10, k=3)
                spline_prediction_othervar = thespline_othervar(x_othervar)
                #map_funcs.xyplot(x_othervar,y_othervar,overlay_x=x_othervar,overlay_y=spline_prediction_othervar,showjupyter=True,dots=True, dotsize=0.01)
                #
                y_unordered_residual = y_unordered_residual - spline_prediction_othervar[order_othervar_togetback]
                print(spline_prediction_othervar.mean(), y_unordered.var(), y_unordered_residual.var())
        #
        x_unordered = traits_matrix[dataindices_touse,par_i]
        order = x_unordered.argsort()
        #
        x = x_unordered[order]
        #
        y_residual = y_unordered_residual[order]
        y = y_unordered[order]
        #
        thespline_residual = interpolate.UnivariateSpline(x, y_residual, s=1e10, k=3)
        spline_prediction_residual = thespline_residual(x)
        #
        map_funcs.xyplot(x,y_residual,overlay_x=x,overlay_y=spline_prediction_residual,showjupyter=True,dots=True, dotsize=0.01)
        #
        minimum_variance_decomposition_data[var_i,par_i] = spline_prediction_residual.var() / y.var()


In [ ]:
### now plot this data as a stemplot
reload(map_funcs)
varnames = ['GPP','LAI','Biomass','Can. Growth Rate','Und. Growth Rate','Can. Mort. Rate','Und. Mort. Rate']

map_funcs.stemplot(potential_variance_decomposition_data, parameter_labels=column_names, variable_labels=varnames, showjupyter=True, file='variance_decomp', parameter_labelsize=.01, variable_labelsize=.01, plot_xmargins=.02, margins_space=.02, maxticks=3, dotsize=0.01, png_dens=300.)



In [ ]:
map_funcs.stemplot(potential_variance_decomposition_data, overlay_data = minimum_variance_decomposition_data, parameter_labels=column_names, variable_labels=varnames, showjupyter=True, file='variance_decomp_potential_minimum', parameter_labelsize=.01, variable_labelsize=.01, plot_xmargins=.02, margins_space=.02, maxticks=3, dotsize=0.01, png_dens=300.)



#####not using this right now: R approach to spline fitting

stats = importr('stats')
base = importr('base')
splines = importr('splines')
SemiPar = importr('SemiPar')

par_indx = 0
var_indx = 0
vars_toassess = [mean_gpp,mean_lai,final_biomass,growthrate_canopy,mortrate_canopy,mortrate_understory]
#
good_data = np.ma.masked_invalid(vars_toassess[var_indx])
dataindices_touse = np.logical_not(np.ma.getmaskarray(good_data))
#
xdata_rvect = robjects.FloatVector(traits_matrix[dataindices_touse,par_indx])
ydata_rvect = robjects.FloatVector(good_data[dataindices_touse])
#
robjects.globalenv["xdata_rvect"] = xdata_rvect
robjects.globalenv["ydata_rvect"] = ydata_rvect
#
spline = SemiPar.spm("ydata_rvect~f(xdata_rvect)")

In [ ]:
traits_matrix[dataindices_touse,par_indx]

## Allocation
#### Plot some ternary diagrams of ecosystem allocation between leaves, wood, and fine roots.  Color the diagrams by some of the more interesting parametric predictors 

In [ ]:
import imp
imp.reload(plt)
import ternary

nppleaf = histfile.variables['NPP_LEAF'][:,startyear*12:endyear*12,0].mean(axis=1)
nppfroot = histfile.variables['NPP_FROOT'][:,startyear*12:endyear*12,0].mean(axis=1)
nppwood = histfile.variables['NPP_CROOT'][:,startyear*12:endyear*12,0].mean(axis=1) + histfile.variables['NPP_STEM'][:,startyear*12:endyear*12,0].mean(axis=1)

nppsum = nppleaf + nppfroot + nppwood

mask = nppsum < nppsum.max()*1e-3
nppleaf = np.ma.masked_array(nppleaf, mask=mask)
nppfroot = np.ma.masked_array(nppfroot, mask=mask)
nppwood = np.ma.masked_array(nppwood, mask=mask)

allocdata = 100. * (np.column_stack([nppleaf, nppwood, nppfroot]).transpose() / (nppsum)).transpose()

# Scatter Plot
size = 6
scale = 100
fontsize=20

interesting_alocation_predictors = [0,4,6,8,9]

for i in range(len(interesting_alocation_predictors)):
    column = interesting_alocation_predictors[i]
    colors = traits_matrix[:,column]

    figure, tax = ternary.figure(scale=scale)
    figure.set_size_inches(1.3*size, size)
    tax.set_title("Allocation diagram, as colored by "+column_names[column], fontsize=fontsize)
    tax.boundary(linewidth=2.0)
    tax.gridlines(multiple=5, color="blue")
    # Plot a few different styles with a legend
    #tax.scatter(allocdata, marker='s', color='red')
    tax.scatter(allocdata, vmax=max(colors),vmin=min(colors),colormap=plt.cm.viridis,colorbar=True,c=colors,cmap=plt.cm.viridis)
    tax.ticks(axis='lbr', linewidth=1, multiple=10)
    tax.left_axis_label("fine root NPP", fontsize=fontsize)
    tax.right_axis_label("wood NPP", fontsize=fontsize)
    tax.bottom_axis_label("leaves NPP", fontsize=fontsize)
    tax.legend()
    ##print(df_traits.shape)
    tax.clear_matplotlib_ticks()
    tax.show()
##print(allocdata[:,2].max(), allocdata[:,2].min())


#### Litterfall.  Compare observations of leaf litterfall to model predictions of leaf allocation (which will be the same as litterfall at steady state).  We can compare seasonality once we add a litterfall flux term to the FATES history output.

In [ ]:
### ingest and plot the Wright et al litterfall data

fname_in = 'benchmark_datasets/LeafMassbySpCensus_20170224.txt'

df = pd.read_csv(fname_in, sep='\t', lineterminator='\r')

nrows = len(df) -1

nspecies = df['sp'].nunique()
ndates = df['YYYY-MM-DD'].nunique()

#print(nspecies)
#print(ndates)

litterfall = np.zeros(ndates)
trap_area = np.zeros(ndates)
dates = []
since_start = np.zeros(ndates)
since_prior = np.ma.masked_all(ndates)

startday = pd.to_datetime('2008-01-01')

for i in range(nrows):
    if df['YYYY-MM-DD'][i] not in dates:
        dates.append(df['YYYY-MM-DD'][i])
    j = dates.index(df['YYYY-MM-DD'][i])
    litterfall[j] = litterfall[j] + df['grams'][i]
    trap_area[j] = trap_area[j] + 0.25
    since_start[j] = (pd.to_datetime(df['YYYY-MM-DD'][i]) - startday).days/ 365.25
    #
    if j > 0:
        since_prior[j] = (pd.to_datetime(df['YYYY-MM-DD'][i]) - pd.to_datetime(dates[j-1])).days/ 365.25

litterfall_rate = litterfall / (trap_area * since_prior)  ### in grams per m2 per year

fig=plt.figure(figsize=(12, 6))
### plot data as continuous timeseries
#plt.plot(since_start, litterfall_rate, marker='', linewidth=0.5, alpha=0.9, color='blue')
#plt.title("Litterfall Rate", loc='left', fontsize=12, fontweight=0)
#plt.xlabel("Time (years since Jan 1 2008)")
#plt.ylabel("Litterfall (g / m2 / yr)")

### plot data as annual cycles superimposed on each other
nyears_litterfall = int(since_start.max() - since_start.min())
for i in range(nyears_litterfall):
    ind_start = np.argmax(since_start>i)
    ind_end = np.argmax(since_start>(i+1))
    plt.plot(since_start[ind_start:ind_end]-i, litterfall_rate[ind_start:ind_end], marker='', linewidth=0.5, alpha=0.9, color='blue')
plt.title("Litterfall Rate", loc='left', fontsize=12, fontweight=0)
plt.xlabel("Time (fraction of year)")
plt.ylabel("Litterfall (g / m2 / yr)")

#### Plot litterfall as a histogram of FATES ensemble member mean flux rates, as compared to the mean of the observations.

In [ ]:
mean_leaf_drymatter_to_leafc = .4482  ## mean of leaf samples in norby dataset

mean_litterfall_cflux = litterfall_rate.mean() * mean_leaf_drymatter_to_leafc * 1e-3

#nppleaf_series = pd.Series(nppleaf, name='NPP leaf')
#sns.distplot(nppleaf, kde=False, rug=False, color="k")

n, bins, patches =plt.hist((nppleaf.flatten(), np.repeat(mean_litterfall_cflux.mean(),40)), histtype="step",bins=30, color=['green','blue'])
plt.xlabel('Leaf C flux (kg C / m2 / yr)')
plt.ylabel('Probability')
plt.title('Leaf C Allocation')

plt.text(0.45, 40, 'Observations', color ='blue', fontsize=15)
plt.text(0.45, 35, 'FATES ensemble members', color ='green', fontsize=15)